# Can we make a change on climate change?
#### EPA1333 Final Assignment
### Introduction
Regardless of the many debates over the impact of humans on climate change, global warming is an observable fact. According to NASA, highest 16 global temperatures have been recorded since 2001, the arctic ice shrank to a minimum in 2012, and carbon dioxide levels in the air are higher than ever!

Data analysis can help us better understand how the climate has changed, why, to make predictions for the future years, and to evaluate measures on how to suppress these changes. In this assignment you receive climate change data from World Bank, and are expected to perform an original and non-trivial analysis using Python.

### Data

#### Climate change indicators
The climate change data offered by World Bank contains a large set of indicators, such as for example CO2 emissions, population growth or renewable energy output. Values of these indicators are available per country and per year. You can download the data in *.csv format from https://data.worldbank.org/topic/climate-change.

In [1]:
# import standard library
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# load world bank data
# skip the unneeded header
wb = pd.read_csv('world_bank/API_19_DS2_en_csv_v2.csv', sep=',', header=0, skiprows=3) 

# read_csv resulted in additional blank series at the last column -> drop the column
wb = wb.drop(wb.columns[[61]], 1)

# display the head of dataframe
wb.head()


,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Aruba,ABW,Urban population (% of total),SP.URB.TOTL.IN.ZS,50.776000,50.761000,50.746000,50.730000,50.715000,50.700000,...,44.147000,43.783000,43.421000,43.059000,42.698000,42.364000,42.058000,41.779000,41.528000,41.304000
1,Aruba,ABW,Urban population,SP.URB.TOTL,27526.000000,28141.000000,28532.000000,28761.000000,28924.000000,29082.000000,...,44686.000000,44375.000000,44052.000000,43778.000000,43575.000000,43456.000000,43398.000000,43365.000000,43331.000000,43296.000000
2,Aruba,ABW,Urban population growth (annual %),SP.URB.GROW,3.117931,2.209658,1.379868,0.799404,0.565140,0.544773,...,-0.435429,-0.698401,-0.730549,-0.623935,-0.464782,-0.273466,-0.133557,-0.076069,-0.078435,-0.080806
3,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.000000,55438.000000,56225.000000,56695.000000,57032.000000,57360.000000,...,101220.000000,101353.000000,101453.000000,101669.000000,102053.000000,102577.000000,103187.000000,103795.000000,104341.000000,104822.000000
4,Aruba,ABW,Population growth (annual %),SP.POP.GROW,3.148037,2.238144,1.409622,0.832453,0.592649,0.573468,...,0.384060,0.131311,0.098616,0.212680,0.376985,0.512145,0.592914,0.587492,0.524658,0.459929


In [3]:
#load country metadata
wb_meta_country = pd.read_csv('world_bank/Metadata_Country_API_19_DS2_en_csv_v2.csv', sep=',', header=0) 

# read_csv resulted in additional blank series at the last column -> drop the column
wb_meta_country = wb_meta_country.drop(wb_meta_country.columns[[5]], 1)

# display the head of dataframe
wb_meta_country.head()


,Country Code,Region,IncomeGroup,SpecialNotes,TableName
0,ABW,Latin America & Caribbean,High income,SNA data for 2000-2011 are updated from offici...,Aruba
1,AFG,South Asia,Low income,Fiscal year end: March 20; reporting period fo...,Afghanistan
2,AGO,Sub-Saharan Africa,Lower middle income,NaN,Angola
3,ALB,Europe & Central Asia,Upper middle income,NaN,Albania
4,AND,Europe & Central Asia,High income,WB-3 code changed from ADO to AND to align wit...,Andorra


In [4]:
#load indicator metadata
wb_meta_indi = pd.read_csv('world_bank/Metadata_Indicator_API_19_DS2_en_csv_v2.csv', sep=',', header=0) 

# read_csv resulted in additional blank series at the last column -> drop the column
wb_meta_indi = wb_meta_indi.drop(wb_meta_indi.columns[[4]], 1)

# display the head of dataframe
wb_meta_indi.head()


,INDICATOR_CODE,INDICATOR_NAME,SOURCE_NOTE,SOURCE_ORGANIZATION
0,SP.URB.TOTL.IN.ZS,Urban population (% of total),Urban population refers to people living in ur...,The United Nations Population Divisions World ...
1,SP.URB.TOTL,Urban population,Urban population refers to people living in ur...,World Bank Staff estimates based on United Nat...
2,SP.URB.GROW,Urban population growth (annual %),Urban population refers to people living in ur...,World Bank Staff estimates based on United Nat...
3,SP.POP.TOTL,"Population, total",Total population is based on the de facto defi...,(1) United Nations Population Division. World ...
4,SP.POP.GROW,Population growth (annual %),Annual population growth rate for year t is th...,Derived from total population. Population sour...


#### Climate data API
The Climate Data API provides programmatic access to most of the climate data used on the World Bank’s Climate Change Knowledge Portal. You can access this data directly from Python using requests. In addition to what was already downloadable as csv data, with this API you are able to access temperature, precipitation and basin level data. Read about it in more detail here: https://datahelpdesk.worldbank.org/knowledgebase/articles/902061-climate-data-api

Below is an example of how to access yearly temperature historical data per country from Python. You can select a country using its ISO aplha3 code: https://unstats.un.org/unsd/methodology/m49/.


In [5]:
import requests
r = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v1/country/cru/tas/year/ROU')
rou = r.json()
rou[:5]

[{'data': 9.215595, 'year': 1901},
 {'data': 8.389345, 'year': 1902},
 {'data': 9.500536, 'year': 1903},
 {'data': 8.901487, 'year': 1904},
 {'data': 9.12619, 'year': 1905}]

As with most data sources, you might find that you data contains missing values. Please handle them appropriately, for example by using interpolation.

#### Other resources
You are encouraged to find more data sources that will make your analysis more meaningful. Please make sure that you document everything carefully. Only use freely available datasets.

#### Assignment
* Create a Jupyter Notebook that contains your explanations and analyses.
* Start the notebook with a clear description of the type of analysis you are going to perform.
* The conceptual contents of the Notebook should be roughly similar to a normal written report of 10-20 pages.
* Some (minimum) properties of the Notebook and your analyses on which we will grade:
    * Required: Combine different data from multiple sources in your analyses.
    * Required: Use multiple types of visualizations of your results.
    * Required: Make sure your Notebook does not generate errors!
    * Required: You should use Python to answer your research questions. Your code should read, clean and format, process and visualize the data. There should be at least some non-trivial processing involved.
    * Whenever possible, make your Notebook read the data directly from the web. This way, your notebook will always use the most up-to-date data available. If not, document carefully where the data was collected what to do when you want to use more up-to-date data.
    * Demonstrate your skills in Python by using typical Python constructs and using the appropriate data structures (lists, dictionaries, tuples, arrays, dataframes, series, recursion, etc.)
    * Write clear, understandable code:
        * Document your code! Put comments when necessary.
        * Use sensible variable names.
        * Break up your code into parts. Use (fruitful) functions.
    * Try to think of reusability of your code. How easy would it be to use your same code if we want to try to do a small variations of your analysis? Can we easily adapt/play around with your code?
    * How difficult were your analyses?
* Make your Notebook self-explanatory. So, it should contain text (with references) as well as your actual analysis code and results.
* If you want to use other libraries for your analyses or visualizations, feel free to do so. However, only use free available and well-known libraries. The ones that come standard with Anaconda are fine. If you want to use something else, that you first need to install, check with us first.
* Create a zip archive and upload it on Brightspace.

#### Example
Here are some example questions you might ask to start the analysis. Please note that this is just an example and that you are expected to come up with your own questions and analyses.
* The EU has the following goal in the Paris agreement: "At least a 40% domestic reduction in greenhouse gases by 2030 compared to 1990 levels.” How is the EU doing at the moment? If they don’t change policy (i.e extrapolation of current trends) where will they end up? [source]
* Can you classify in good/neutral/bad countries?
* What is the trend per continent with respect to gas emissions? How do countries within the same continent behave? Is it fair to make statements over whole continents?
* Where do you find the highest increase in temperatures compared to 1960?
* Suppose that each country has a % growth or reduction of CO2, where do we end up?
* What are the countries with most gas emissions? How does that change when you normalize by their size / population?
* What is the country with the best trend in renewable energy over the last 10 years?

In [7]:
# load world bank data
country_class_y = pd.read_csv('world_bank/CCPI_2017.csv', sep=',', header=0) 
country_class_y.head()

,Countries,Label
0,France,Good
1,Sweden,Good
2,United Kingdom,Good
3,Cyprus,Good
4,Luxembourg,Good


In [8]:
wb_meta_country[['Country Code', 'TableName']].head()

,Country Code,TableName
0,ABW,Aruba
1,AFG,Afghanistan
2,AGO,Angola
3,ALB,Albania
4,AND,Andorra


In [9]:
df = pd.DataFrame()
for country in country_class_y['Countries']:
    #print(country)
    df1 = wb[wb['Country Name'] == country]
    df = pd.concat( [df,df1], ignore_index=True, axis=0)
        

In [12]:
df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,France,FRA,Urban population (% of total),SP.URB.TOTL.IN.ZS,6.188000e+01,6.260700e+01,6.348900e+01,6.470200e+01,6.589800e+01,6.707100e+01,...,7.762100e+01,7.786400e+01,7.810600e+01,7.834500e+01,7.858400e+01,7.882000e+01,7.905500e+01,7.928900e+01,7.952000e+01,7.975000e+01
1,France,FRA,Urban population,SP.URB.TOTL,2.896865e+07,2.970374e+07,3.055068e+07,3.157696e+07,3.258617e+07,3.355144e+07,...,4.969004e+07,5.012494e+07,5.054008e+07,5.094580e+07,5.134897e+07,5.175305e+07,5.217517e+07,5.259394e+07,5.297946e+07,5.334965e+07
2,France,FRA,Urban population growth (annual %),SP.URB.GROW,2.424503e+00,2.505858e+00,2.811428e+00,3.304057e+00,3.146026e+00,2.919203e+00,...,9.335545e-01,8.714278e-01,8.248035e-01,7.995637e-01,7.882418e-01,7.838471e-01,8.123421e-01,7.994288e-01,7.303273e-01,6.963089e-01
3,France,FRA,"Population, total",SP.POP.TOTL,4.681424e+07,4.744475e+07,4.811965e+07,4.880368e+07,4.944940e+07,5.002377e+07,...,6.401623e+07,6.437499e+07,6.470704e+07,6.502751e+07,6.534278e+07,6.565979e+07,6.599857e+07,6.633196e+07,6.662407e+07,6.689611e+07
4,France,FRA,Population growth (annual %),SP.POP.GROW,1.229610e+00,1.337853e+00,1.412470e+00,1.411512e+00,1.314427e+00,1.154839e+00,...,6.187115e-01,5.588574e-01,5.144864e-01,4.940375e-01,4.836449e-01,4.839823e-01,5.146361e-01,5.038712e-01,4.394107e-01,4.074910e-01


In [13]:
df.shape

(4480, 61)

In [19]:
df_2000 = df.iloc[:, [0,2,49]]
df_2000.head()

,Country Name,Indicator Name,2005
0,France,Urban population (% of total),7.713000e+01
1,France,Urban population,4.873024e+07
2,France,Urban population growth (annual %),1.074557e+00
3,France,"Population, total",6.317936e+07
4,France,Population growth (annual %),7.538056e-01


In [ ]:
#df[['label']]
country_class_y.columns = ['Country Name', 'Label']
country_class_y.head()

In [ ]:
df_2000_label = pd.merge(df_2000, country_class_y, on="Country Name", how="outer" )
df_2000_label.head()
df_2000_label.shape

In [ ]:
# preprocessing data
df_2000_label_good = df_2000_label[(df_2000_label['Indicator Name'] == 'Urban population (% of total)') & (df_2000_label['Label'] == 'Good')]

df_2000_label_good

In [ ]:
len(wb_meta_indi['INDICATOR_NAME'])

In [ ]:
list_deleted_indi = []
list_saved_indi = []
df_test = pd.DataFrame()

for label in country_class_y['Label'].unique():
    for indicator in wb_meta_indi['INDICATOR_NAME']:
        df_temp = pd.DataFrame()
        df_temp = df_2000_label[(df_2000_label['Indicator Name'] == indicator) & (df_2000_label['Label'] == label)]
        
        # df_temp = df_temp.fillna(df_temp.mean())
        df_temp.interpolate(method=’polynomial’, order=4)
        print(df_temp)
        
        if(df_temp['2000'].isnull().values.all()):
            list_deleted_indi.append(indicator) 
        else:
            list_saved_indi.append(indicator) 
            
        df_test = pd.concat([df_test,df_temp], ignore_index=True, axis=0)
        
print(list_deleted_indi)
df_test

In [ ]:
len(list_deleted_indi)

In [ ]:
len(list_saved_indi)

In [ ]:
set_saved_indi_filter = set(wb_meta_indi['INDICATOR_NAME']) - set(list_deleted_indi)

In [ ]:
len(set_saved_indi_filter)

In [ ]:
df_test.isnull().values.any()

In [ ]:
df_test[df_test['Indicator Name'] == 'Urban population (% of total)']
df_test.shape

In [ ]:
df_test_clean = pd.DataFrame()
for indi in set_saved_indi_filter:
    df_temp = df_test[df_test['Indicator Name'] == indi]
    df_test_clean = pd.concat([df_test_clean, df_temp], ignore_index = True, axis = 0)

print(df_test_clean.shape)
df_test_clean.head()

In [ ]:
3584/56

In [ ]:
# reshape the matrix feature
col = list(set_saved_indi_filter)
ind = list(country_class_y['Country Name'])

Xtrain = pd.DataFrame(np.zeros(shape = (56, 64)), columns = col, index = ind)
for country in country_class_y['Country Name']:
    Xtrain.loc[country] = df_test_clean[df_test_clean['Country Name'] == country]['2000'].values
Xtrain.shape

In [ ]:
ytrain = country_class_y['Label']
ytrain.head()

In [ ]:
from sklearn.svm import SVC
svc = SVC(kernel='rbf', class_weight='balanced')

In [ ]:
svc.get_params().keys()

In [ ]:
from sklearn.grid_search import GridSearchCV
param_grid = {'C': [0.05, 0.1, 0.5, 1, 5, 10, 50], 'gamma': [1E-20 , 1E-15, 1E-12, 1E-10, 0.00001]}
grid = GridSearchCV(svc, param_grid)

%time grid.fit(Xtrain, ytrain)
print(grid.best_params_)

In [ ]:
model = grid.best_estimator_


In [ ]:
wb_test = wb[['Country Name','Indicator Name','2000']]
wb_test.head()

In [ ]:
# Xtest

df_test_set = pd.DataFrame()
for indi in set_saved_indi_filter:
    df_temp = wb_test[wb_test['Indicator Name'] == indi]
    df_test_set = pd.concat([df_test_set, df_temp], ignore_index = True, axis = 0)

print(df_test_set.shape)
df_test_set.head()


In [ ]:
len(wb['Country Name'].unique())

In [ ]:
16896/264

In [ ]:
set_test = set(wb['Country Name']) - set(country_class_y['Country Name'])

df_test_set_country = pd.DataFrame()
for country in set_test:
    # print(country)
    df_temp = df_test_set[df_test_set['Country Name'] == country]
    df_test_set_country = pd.concat([df_test_set_country, df_temp], ignore_index = True, axis = 0)

In [ ]:
len(set_test)

In [ ]:
df_test_set_country.head()

In [ ]:
# reshape the matrix feature
col = list(set_saved_indi_filter)
ind = list(country_class_y['Country Name'])

for country in country_class_y['Country Name']:
    Xtrain.loc[country] = df_test_clean[df_test_clean['Country Name'] == country]['2000'].values
Xtrain.shape

Xtrain = pd.DataFrame(np.zeros(shape = (56, 64)), columns = col, index = ind)
for country in set_test:
    df_test_country[]
    

In [ ]:
df_vietnam = df_test_set_country[df_test_set_country['Country Name'] == 'Vietnam']
df_vietnam = df_vietnam.fillna(0)

In [ ]:
df_vietnam.isnull().sum()

In [ ]:
Xtest = pd.DataFrame(df_vietnam['2000'].values.reshape(1,64))

In [ ]:
ytest = model.predict(Xtest)
ytest

In [ ]:
from sklearn.cross_validation import cross_val_score
cross_val_score(model, Xtrain, ytrain, cv=4)

In [ ]:
list_deleted_indi = []
df_test = pd.DataFrame()

for label in country_class_y['Label'].unique():
    for indicator in wb_meta_indi['INDICATOR_NAME']:
        df_temp = pd.DataFrame()
        df_temp = df_2000[(df_2000['Indicator Name'] == indicator) & (df_2000_label['Label'] == label)]
        print(df_temp)
        #df_temp = df_temp.fillna(df_temp.mean())
        
        if(df_temp['2000'].isnull().values.all()):
            #print(df_temp)
            list_deleted_indi.append(indicator) 
        
        if((not df_temp['2000'].isnull().values.all()) & df_temp.isnull().values.any()):
            df_temp = df_temp.fillna(df_temp.mean())
            df_test = pd.concat( [df_test,df_temp], ignore_index=True, axis=0)
            print(df_temp)
            break
        
        df_test = pd.concat( [df_test,df_temp], ignore_index=True, axis=0)
        
    if((not df_temp['2000'].isnull().values.all()) & df_temp.isnull().values.any()):
        break

print(list_deleted_indi)
df_test